# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [36]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""
link_system_prompt += """Example 1
Input:
https://acme.com/
https://acme.com/about
https://acme.com/careers
https://acme.com/blog
"""
link_system_prompt += """
Output:
{
  "links": [
    {"type": "about page", "url": "https://acme.com/about"},
    {"type": "careers page", "url": "https://acme.com/careers"}
  ]
}
"""
link_system_prompt += """
Example 2
Input:
https://example.org/blog
https://example.org/contact

Output:
{
  "links": []
}
"""

In [34]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
You should respond in text string in this example:
{
    "links": [
        {"type": "news", "url": "this is a text"}
    ]
}



In [28]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [29]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [30]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [31]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/SRPO',
 '/baidu/ERNIE-4.5-21B-A3B-Thinking',
 '/Qwen/Qwen3-Next-80B-A3B-Instruct',
 '/Qwen/Qwen3-Next-80B-A3B-Thinking',
 '/google/vaultgemma-1b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/multimodalart/wan-2-2-first-last-frame',
 '/spaces/IndexTeam/IndexTTS-2-Demo',
 '/spaces/tencent/HunyuanImage-2.1',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/InternRobotics/OmniWorld',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/W

In [37]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/SRPO
Updated
1 day ago
•
3.08k
•
743
baidu/ERNIE-4.5-21B-A3B-Thinking
Updated
5 days ago
•
102k
•
702
tencent/HunyuanImage-2.1
Updated
4 days ago
•
5.53k
•
599
Qwen/Qwen3-Next-80B-A3B-Instruct
Updated
about 23 hours ago
•
198k
•
528
Qwen/Qwen3-Next-80B-A3B-Thinking
Updated
about 23 hours

In [48]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown. \
Include details of company culture, customers and careers/jobs if you have the information."


In [49]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [50]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/SRPO\nUpdated\n2 days ago\n•\n3.61k\n•\n784\nbaidu/ERNIE-4.5-21B-A3B-Thinking\nUpdated\nabout 4 hours ago\n•\n112k\n•\n718\nQwen/Qwen3-Next-80B-A3B-Instruct\nUpdated\nabout 1 hour ago\n•\n305k\n•\n611\nQwen/Qwen3-Next-80B-A3B-Thinking\nUpdated\n2 days ago\n•\n160k\n•\n351\ngoogle/vaultgemma-1b\nUpdated\n5 days ago\n•\n1.28k\n•\n311\nBrowse 1M+ models\nSpaces\nRunning\n13.6k\n13.6k\

In [51]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [52]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face! 🎉

**The AI community building the future (and hugging it too)!**

---

## Who We Are 🤔

At Hugging Face, we're not just an AI platform; we're an **AI family**! Like a cozy community gathering where everyone brings their favorite model, dataset, or snack. We host 1 million+ models that make each algorithm feel like the star of the show!

- **Chatbots** that are better listeners than your therapist. 
- **Hidden talents** inside datasets (yes, your dusty numbers can truly shine).
- **Spaces** where AI applications practically throw dance parties every day.

---

## Our Customers: Who's Using Hugging Face? 🎉

More than **50,000 organizations** are rocking with us! From **Google** to **Microsoft**, even **Grammarly** decided to join the fun because who doesn't want impeccable grammar in their datasets?  

Big names in AI management? We've got you covered:
- **AI at Meta**: We spread the love.
- **Amazon**: Well, who wouldn’t want a little bit of AI magic with their deliveries?

---

## Our Culture: Work Hard, Hug Harder! 🤗

At Hugging Face, we give hugging a new meaning. We believe in:
- **Collaboration** over competition (because sharing is caring).
- **Open Source** community vibes (together we can conquer the ML world).
- The freedom to express ideas, whether it's through cutting-edge models or a really good meme.

Open door policy? More like an **open virtual hugging policy**! 😂

---

## Careers: Join the Hug Squad! 🚀

Are you ready to jump into the world of AI with knowledge and fun? Here's why you should consider a career with us:
- **Innovative projects** that even your wildest dreams couldn’t foresee!
- A chance to contribute to the **next big thing** in technology, while also having a lot of fun.
- Enrich your career path in one of the fastest-growing fields, while hugging it out with others who share your love for AI!

We've got positions for dreamers, doers, and everything in between. 

---

So, are you ready to embrace our AI magic? Whether you're looking to collaborate, learn, or just give us a big hug, Hugging Face is the place for you!  

Grab your hugging gear, and let’s make the future together! 💪🤖✨

---

## Get in Touch 🌐

[Website](https://huggingface.co) | [Twitter](https://twitter.com/huggingface) | [GitHub](https://github.com/huggingface)

**Remember**: No hug is too big and no model is too small. Welcome to the Hugging Face family!

In [59]:
translate_system_prompt = """You are provided with text written in English, formatted in Markdown. \
You are a Spanish translator, you translate the given text into Spanish.\
Respond in markdown.
"""

In [60]:
def translate_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages for brochure; .\n"
    user_prompt += get_brochure_user_prompt(company_name, url)
    user_prompt += f"All in English\n"
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [61]:
def translate_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content": translate_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [62]:
translate_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face – La comunidad de IA que construye el futuro

**Hugging Face** es una plataforma donde la comunidad de aprendizaje automático colabora en modelos, conjuntos de datos y aplicaciones. Explora aplicaciones de IA o navega entre más de 1 millón de modelos.

## Contenidos

- **Modelos**
- **Conjuntos de Datos**
- **Espacios**
- **Comunidad**
- **Documentación**
- **Empresa**
- **Precios**
- **Iniciar sesión**
- **Registrarse**

## Lo más destacado esta semana

### Modelos Trending
- **tencent/SRPO**
  - Actualizado: hace 2 días
  - ⭐️ 3.61k ⭐️ 
  - 💬 784 comentarios
  
- **baidu/ERNIE-4.5-21B-A3B-Thinking**
  - Actualizado: hace aproximadamente 5 horas
  - ⭐️ 112k ⭐️ 
  - 💬 718 comentarios
  
- **Qwen/Qwen3-Next-80B-A3B-Instruct**
  - Actualizado: hace aproximadamente 1 hora
  - ⭐️ 305k ⭐️ 
  - 💬 611 comentarios

## Espacios

### Running
- **DeepSite v2**
  - Genera cualquier aplicación con DeepSeek.
  
- **Wan2.2 14B Fast**
  - Generar un video a partir de una imagen con un aviso de texto.

## Conjuntos de Datos Destacados
- **HuggingFaceFW/finepdfs**
  - Actualizado: hace 9 días
  - ⭐️ 57.2k ⭐️ 
  - 💬 497 comentarios

- **LucasFang/FLUX-Reason-6M**
  - Actualizado: hace 5 días
  - ⭐️ 14.6k ⭐️ 
  - 💬 53 comentarios

## Herramientas de Machine Learning

### Acelera tu aprendizaje automático
Proporcionamos soluciones de cómputo y empresariales. Despliega aplicaciones en puntos de inferencia optimizados o actualiza tus aplicaciones en Espacios a una GPU con unos pocos clics.

### Precios
- A partir de $0.60/hora para GPU.
- Equipo y empresa: A partir de $20/usuario/mes.

## Más de 50,000 organizaciones utilizan Hugging Face
Algunas de las organizaciones destacadas incluyen:
- Meta
- Amazon
- Google
- Microsoft

## Nuestro código abierto
Estamos construyendo la base de herramientas de ML con la comunidad. 

### Principales proyectos
- **Transformers**: Modelos de IA de última generación para PyTorch.
- **Diffusers**: Modelos de difusión de última generación en PyTorch.

## Conéctate con nosotros
- **GitHub**
- **Twitter**
- **LinkedIn**
- **Discord**

¡Únete a la comunidad de Hugging Face y contribuye a construir el futuro de la inteligencia artificial!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
Hugging Face is a collaborative platform designed for the machine learning community to share, develop, and improve on models, datasets, and applications. With over 1 million models and 250,000 datasets, our platform is at the forefront of advancing AI technology.

## Our Offerings
- **Models**: Explore an extensive collection of cutting-edge AI models, including state-of-the-art architectures.
- **Datasets**: Access and engage with a diverse range of datasets suitable for various ML tasks.
- **Spaces**: Collaborate on applications in a vibrant ecosystem where sharing and creativity thrive.

### Trusted by Industry Leaders
Join more than 50,000 organizations, including:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

## Company Culture
At Hugging Face, we believe in the power of community and open-source collaboration. Our culture is defined by:
- **Inclusivity**: We welcome contributions from people of all backgrounds, empowering everyone to participate in the ML revolution.
- **Innovation**: We encourage experimentation and exploration, allowing our team and users to push the boundaries of what's possible.
- **Transparency**: As an open-source leader, we prioritize clear communication and collective progress within our community.

## Career Opportunities
Join our growing team in an environment that fosters creativity and growth! We are actively looking for passionate individuals in various roles. Explore our [Current Openings](#) and be part of a team that’s making a difference in AI and ML.

### Why Work With Us?
- Work on groundbreaking AI projects with industry experts.
- Contribute to an open-source platform that impacts millions of developers and researchers.
- Enjoy a flexible work culture that values balance and innovation.

## Get Involved
Become part of the conversation! Engage with us on social platforms such as GitHub, Twitter, LinkedIn, and Discord. Whether you’re a seasoned AI professional or a budding enthusiast, your journey in machine learning starts here at Hugging Face.

---

Together, let's shape the future of AI! For more information, visit our website and discover how you can contribute.

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# 🐻‍❄️ Hugging Face Brochure: The AI Community That Hugs Back! 🤗

### Welcome to Hugging Face: Building the AI Future (with a Smile!)

Are you ready to join a community that has more models than a fashion show? Welcome to Hugging Face, where we collaborate, create, and caffeinate the world of Machine Learning (ML)! Our platform is where AI enthusiasts—both serious scientists and emoji-loving artists—come together to share **models, datasets, and probably a few dad jokes**.

---

## 🤖 What Do We Do? 

- **Models Galore**: Looking for a model? We’ve got 1 million+ just waiting for you! From predictive text to high-quality image generation, our models are like a Swiss Army knife for AI. Just remember, they don’t make coffee…yet! ☕  
- **Datasets**: With over 250,000 datasets, you'll never be short on data to munch on. Got something specific in mind? We likely have it—if not, just train your own “Fluffy Model 5000” with our awesome resources! 📊  
- **Spaces**: Not just for lounging! Our spaces let you run and collaborate on 400,000+ applications. Think of them as virtual playgrounds where creativity blooms! 🌻

---

## 🎉 The Hugging Community 

At Hugging Face, we’re proud to count over **50,000 organizations** as part of our family. From Amazon to your quirky indie startup (yes, we mean you!), everyone loves sharing their AI journey. So why not join us? 

And don't worry—we won't ghost you after you sign up! We're all about community engagement. Chat with fellow ML aspirants and drop your favorite memes or just hang out in our Discord!

---

## 👩‍💻 Careers at Hugging Face 

Want to work with some of the brightest minds and also indulge in **AI banter**? Check out our career options! We’re always on the lookout for:

- **AI Whisperers**: Those who understand machine learning as well as they understand their morning coffee order.
- **Data Enthusiasts**: If you've ever touched a dataset and felt a tingling sensation, we need you!
- **Creative Coders**: If coding makes you dance, bring your moves to our team!

Perks? You bet! From **flexible hours** to a work culture that promotes hugging (well, metaphorically!), you’ll feel right at home. 

---

## 🚀 Jump Onboard!

### Why Hugging Face? Because we promise:

- **Immediate Puns**: Let’s face it, who doesn’t love a good pun during meetings?
- **Supportive Vibes**: Whether it’s AI queries or just wanting to grab coffee, we’re like that super supportive friend you wish you had. 
- **Constant Growth**: Be part of the journey in building state-of-the-art AI models and tools. Your contributions can be the next big thing in tech!

---

So, whether you’re interested in AI, seeking a collaborative community, or are just really into hugging (virtually or otherwise), Hop on the Hugging Face train! 🚂🤗

### Join Us Today!
[Sign Up Now!](#) and embrace the future with Hugging Face! 

*(Disclaimer: We cannot guarantee actual hugs, but emotional ones are more than welcome!)*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>